In [1]:
print("ok")

ok


In [2]:
import os

In [3]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2


In [4]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [5]:
key = os.getenv("OPENAI_API_KEY")

In [6]:
llmobj=ChatOpenAI(openai_api_key = key, model = "gpt-3.5-turbo", temperature=0.7)

/Users/mayankchugh/gitRepos/mayankchugh.learning/MCQ-Generator/mcqvenv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
%pwd

'/Users/mayankchugh/gitRepos/mayankchugh.learning/MCQ-Generator/experiment'

In [8]:
projectPath = os.getenv("PATH_TO_PROJECT")
projectPath

'/Users/mayankchugh/gitRepos/mayankchugh.learning/MCQ-Generator/'

In [9]:
mcqPrjPath = os.getenv("MCQ_PROJECT_FOLDER")
mcqPrjPath

'MCQ-Generator/'

In [10]:
with open(projectPath+"Response.json","r") as f:
    RESPONSE_JSON=json.load(f)

In [11]:
print(RESPONSE_JSON)

{'1': {'no': '1', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '2': {'no': '2', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '3': {'no': '3', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}}


In [12]:
TEMPLATE="""Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}

"""

In [13]:
quiz_generation_prompt = PromptTemplate (
    input_variables=["text","number","subject","tone","RESPONSE_JSON"],
    template=TEMPLATE
)

In [14]:
quiz_chain=LLMChain(llm=llmobj,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [15]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [16]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [17]:
review_chain = LLMChain(llm=llmobj,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [18]:
generate_evaluate_chain = SequentialChain(
    chains= [quiz_chain,review_chain], 
    input_variables=["text","number","subject","tone","RESPONSE_JSON"], 
    output_variables=["quiz", "review"], 
    verbose=True)

In [19]:
DATA_PATH=projectPath+"data.txt"
DATA_PATH

'/Users/mayankchugh/gitRepos/mayankchugh.learning/MCQ-Generator/data.txt'

In [20]:
with open(DATA_PATH,"r") as file:
    TEXT=file.read()

In [21]:
print(TEXT)

Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:

Systems, architecture and infrastructure for Generative AI
Machine learning and Modeling using LLMs and Diffusion models
Large Language Models and its applications
Multi-modal Generative AI and its applications
Gen AI based Plugins and

TEXT

NUMBER=5

SUBJECT="AI"

TONE="Simple"  

RESPONSE_JSON=RESPONSE_JSON

In [ ]:
with get_openai_callback() as cb:
    generate_evaluate_chain(

        "text":TEXT,
        "number":NUMBER,
        "subject":SUBJECT,
        "tone":TONE,
        "RESPONSE_JSON":RESPONSE_JSON

    )